In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
MSAI = ["CMPE 252", "CMPE 257", "ISE 201", "CMPE 255", "CMPE 256", "CMPE 259", "CMPE 249", "CMPE 258", "CMPE 260", "CMPE 214", "CMPE 217", "CMPE 243", "CMPE 266", "CMPE 281", "CMPE 297", "CMPE 298", "CMPE 294", "CMPE 299A", "CMPE 295A"]
MSCMPE = ["CMPE 200", "CMPE 220", "CMPE 240", "CMPE 213", "CMPE 214", "CMPE 249", "CMPE 281", "CMPE 283", "CMPE 206", "CMPE 207", "CMPE 208", "CMPE 209", "CMPE 210", "CMPE 242", "CMPE 243", "CMPE 244", "CMPE 245", "CMPE 265", "CMPE 209", "CMPE 211", "CMPE 219", "CMPE 279", "CMPE 287", "CMPE 252", "CMPE 255", "CMPE 256", "CMPE 257", "CMPE 258", "CMPE 260", "CMPE 294", "CMPE 299A", "CMPE 295A"]
MSSE = ["CMPE 272", "CMPE 202", "CMPE 255", "CMPE 273", "CMPE 275", "CMPE 281","CMPE 283", "CMPE 285", "CMPE 287", "CMPE 206", "CMPE 207", "CMPE 257", "CMPE 258", "CMPE 209", "CMPE 279", "CMPE 294", "CMPE 299A", "CMPE 295A"]

In [ ]:
MAJORS = [MSAI, MSCMPE, MSSE]

In [ ]:
MSE = set(MSAI + MSCMPE + MSSE)
df = pd.DataFrame(columns=list(MSE))
df

,CMPE 295A,CMPE 260,CMPE 219,CMPE 283,CMPE 275,CMPE 202,CMPE 297,CMPE 249,CMPE 279,CMPE 252,...,CMPE 259,CMPE 217,CMPE 281,CMPE 258,CMPE 210,CMPE 299A,CMPE 208,CMPE 200,CMPE 294,CMPE 242


In [ ]:
dictionary = {}
for i in range(500):
  for C in MSAI:
    dictionary[C] = np.random.randint(0,2)
  df = pd.concat([df, pd.DataFrame([dictionary])], ignore_index=True)

  dictionary = {}
  for C in MSCMPE:
      dictionary[C] = np.random.randint(0,2)
  df = pd.concat([df, pd.DataFrame([dictionary])], ignore_index=True)

  dictionary = {}
  for C in MSSE:
      dictionary[C] = np.random.randint(0,2)
  df = pd.concat([df, pd.DataFrame([dictionary])], ignore_index=True)
df.fillna(0, inplace=True)
df

<ipython-input-93-dab6f2d5cf69>:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


,CMPE 295A,CMPE 260,CMPE 219,CMPE 283,CMPE 275,CMPE 202,CMPE 297,CMPE 249,CMPE 279,CMPE 252,...,CMPE 259,CMPE 217,CMPE 281,CMPE 258,CMPE 210,CMPE 299A,CMPE 208,CMPE 200,CMPE 294,CMPE 242
0,1,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,1,1,1
2,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,1,0,0,0,1,...,0,1,1,0,0,1,0,0,1,0
4,1,1,0,1,0,0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,1,0,1,0,0,0,0,0,0,1,...,0,0,0,1,1,0,1,0,0,1
1496,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1497,1,0,0,0,0,1,1,1,1,1,...,0,1,1,0,0,0,0,0,1,0
1498,0,1,1,0,0,0,0,1,0,1,...,0,0,1,0,1,1,0,0,1,1


In [ ]:
def find_major(MAJORS, Course_List):
  index = 0
  max_count = 0
  for m in MAJORS:
    count = 0
    for c in Course_List:
      if c in m:
        count += 1
    if count > max_count:
      max_count = count
      index = MAJORS.index(m)
  return index

def top_neighbors(index, df, neighbors=5):
  jaccard_similarity = []
  for i in range(0, len(df) - 1):
    jaccard_similarity.append(jaccard_score(df[MAJORS[index]].iloc[-1], df[MAJORS[index]].iloc[i]))

  top = []
  while len(top) < neighbors:
    max_val = jaccard_similarity.index(max(jaccard_similarity))
    top.append(max_val)
    jaccard_similarity[max_val] = -1
  return top

def reccomend_coures(file, MAJORS, df):
  transcript = pd.read_html("/content/ps.xls", header=None)[0]
  transcript.drop(columns=['Description','Term', 'Grade', 'Units', 'Grd Points', 'Repeat Code','Reqmnt Desig', 'Status'], inplace=True)
  Course_List = transcript['Course'].tolist()

  Courses = {}
  for i in transcript['Course']:
    Courses[i] = 1
  df = pd.concat([df, pd.DataFrame([Courses])], ignore_index=True).fillna(0)

  index = find_major(MAJORS, Course_List)
  top = top_neighbors(index, df)

  cols  = []
  for t in top:
    for column in df[MAJORS[index]].columns:
      if df.iloc[-1][column] == 0 and df.iloc[t][column] == 1:
        cols.append(column)

  return set(cols)

In [ ]:
file  = "/content/ps.xls"
courses = reccomend_coures(file, MAJORS, df)
print(courses)

<ipython-input-95-be1c6224eb3e>:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.concat([df, pd.DataFrame([Courses])], ignore_index=True).fillna(0)


{'CMPE 243', 'CMPE 217', 'CMPE 281', 'CMPE 214', 'CMPE 297', 'CMPE 249'}
